# [10] Segmentation 신경망 만들기

본 실습에서는 바이오 메디컬 이미지 세그멘테이션에서 가장 대중적으로 사용되는 `U-Net`을 구현해보겠습니다.

`U-Net`은 비단 메디컬 이미지 분야 뿐만 아니라 날씨 예측 등 많은 곳에서 회자되는 네트워크 구조입니다.

End-to-End로 Segmentation하는 심플하고 효과적인 방법이기도 합니다.

https://arxiv.org/abs/1505.04597 : 본 논문의 링크입니다

## 왜 U-Net인가?

네트워크 구성의 형태 (`U`)로 인해 U-Net 이라는 이름이 붙여졌습니다.

![U_net_overview](./imgs/U_Net_overview.png)

U-Net은 이미지의 다양한 컨텍스트 (특징) 정보를 얻기 위한 부분과 Localization (지역화)를 위한 부분이 대칭을 이루어 붙여진 형태입니다.

아래의 자료는 https://medium.com/@msmapark2/u-net-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-u-net-convolutional-networks-for-biomedical-image-segmentation-456d6901b28a 를 참고하였습니다.

![contracting_path](./imgs/contracting_path.png)

![expanding_path](./imgs/expanding_path.png)

![skip_architecture](./imgs/skip_architecture.png)

In [2]:
import torch
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F

In [3]:
class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=(3,3),padding=1):
        super(ConvBlock,self).__init__()
        self.conv = ??????????
        self.batchnorm = ??????
        self.relu = ?????????
        
    def forward(self,x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

In [4]:
class StackEncoder(nn.Module):
    def __init__(self,channel1,channel2,kernel_size=(3,3),padding=1):
        super(StackEncoder,self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.block = nn.Sequential(
            ?????????,
            ?????????,     
        )
        
    def forward(self,x):
        copy_out = self.block(x)
        poolout = self.maxpool(copy_out)
        return copy_out,poolout

In [5]:
class StackDecoder(nn.Module):
    def __init__(self,copy_channel,channel1,channel2,kernel_size=(3,3),padding=1):
        super(StackDecoder,self).__init__()
        self.unConv = nn.ConvTranspose2d(channel1,channel1,kernel_size=(2,2),stride=2)
        self.block = nn.Sequential(
            ConvBlock(?????????,?????????,kernel_size,padding),
            ConvBlock(?????????,?????????,kernel_size,padding),
            ConvBlock(?????????,?????????,kernel_size,padding),
        )
        
    def forward(self,x,down_copy):
            _, channels, height, width = down_copy.size()  
            x = self.unConv(x)
            x = torch.cat([?????????, ?????????], 1)
            x = self.block(x)
            return x

In [6]:
class Unet(nn.Module):
    def __init__(self):
        super(Unet,self).__init__()
        
        self.down1 = ?????????(3,32,kernel_size=(3,3))             
        self.down2 = ?????????(32,64,kernel_size=(3,3))            
        self.down3 = ?????????(64,128,kernel_size=(3,3))           
        self.down4 = ?????????(128,256,kernel_size=(3,3))          
        
        self.center = ?????????(256,256,kernel_size=(3,3),padding=1)  
        
        self.up4 = ?????????(256,256,128,kernel_size=(3,3))        
        self.up3 = ?????????(128,128,64,kernel_size=(3,3))         
        self.up2 = ?????????(64,64,32,kernel_size=(3,3))           
        self.up1 = ?????????(32,32,16,kernel_size=(3,3))           
        self.conv = Conv2d(16,3,kernel_size=(1,1),bias=True)
        
    def forward(self,x):
        copy1,out = self.down1(x)  
        copy2,out = self.down2(out)  
        copy3,out = self.down3(out)
        copy4,out = self.down4(out)
        
        out = self.center(out)
        
        up4 = self.up4(out,copy4)
        up3 = self.up3(up4,copy3)
        up2 = self.up2(up3,copy2)
        up1 = self.up1(up2,copy1)
        
        out = self.conv(up1)
        out = nn.?????????()(out)


        return out